In [5]:
import pandas as pd

df = pd.read_csv("../data/raw/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv")

In [8]:
df_1 = df.copy()
df_1.columns = df_1.columns.str.replace('W', 'A')
df_1.columns = df_1.columns.str.replace('L', 'B')
df_1['is_AWin'] = 1

df_2 = df.copy()
df_2.columns = df_2.columns.str.replace('W', 'B')
df_2.columns = df_2.columns.str.replace('L', 'A')
df_2['is_AWin'] = 0

In [15]:
base_df = pd.concat([df_1, df_2])[['Season', 'ATeamID', 'BTeamID', 'is_AWin']]

In [16]:
df_seeds = pd.read_csv("../data/raw/MDataFiles_Stage1/MNCAATourneySeeds.csv")

In [17]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_A = df_seeds[['Season','TeamID','seed']].copy()
seeds_B = df_seeds[['Season','TeamID','seed']].copy()
seeds_A.columns = ['Season','ATeamID','A_seed']
seeds_B.columns = ['Season','BTeamID','B_seed']
base_df = pd.merge(base_df, seeds_A,on = ["Season","ATeamID"],how = "left")
base_df = pd.merge(base_df, seeds_B,on = ["Season","BTeamID"],how = "left")
base_df["seeddiff"] = base_df["A_seed"] - base_df["B_seed"]

In [18]:
train_df = base_df.loc[
    base_df['Season'] < 2016,
]

In [19]:
train_X = train_df[['A_seed', 'B_seed', 'seeddiff']]
train_y = train_df['is_AWin']

In [20]:
import lightgbm as lgbm

clf = lgbm.LGBMClassifier()
clf.fit(train_X, train_y)

LGBMClassifier()

In [21]:
ssub_df = pd.read_csv("../data/raw/MDataFiles_Stage1/MSampleSubmissionStage1.csv")

In [25]:
sub_df = ssub_df['ID'].str.split(pat='_', expand=True).astype(int)
sub_df.columns = ['Season', 'ATeamID', 'BTeamID']

sub_df = pd.merge(sub_df, seeds_A,on = ["Season","ATeamID"],how = "left")
sub_df = pd.merge(sub_df, seeds_B,on = ["Season","BTeamID"],how = "left")
sub_df["seeddiff"] = sub_df["A_seed"] - sub_df["B_seed"]

In [35]:
test_X = sub_df[['A_seed', 'B_seed', 'seeddiff']]
prediction = clf.predict_proba(test_X)[:, 1]

ssub_df['Pred'] = prediction

In [36]:
ssub_df.to_csv('initial_submission.csv', index=False)